# ИНДЕКС ПОТРЕБИТЕЛЬСКОЙ ЛОЯЛЬНОСТИ (NPS) российской telecomm-компании

## Краткое описание проекта

В фокусе исследования - изучение текущего уровеня потребительской лояльности (NPS) телекоммуникационной компании среди клиентов из России.   
__Основная цель - построение дашборда, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков.__  

__Чтобы определить уровень лояльности, клиентам задавали классический вопрос:__ «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».  
Оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).
  
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису).

В работе есть следующие таблицы:  
- таблица user - содержит основную информацию о клиентах;
- таблица location - справочник территорий, в которых телеком-компания оказывает услуги;
- таблица age_segment - данные о возрастных сегментах клиентов;
- таблица traffic_segment - данные о выделяемых сегментах по объёму потребляемого трафика;
- таблица lifetime_segment - данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.  

Необходимо объединить все таблицы, проверить данные и выгрузить для работы в Tableu.

## Изучение данных

Извлечём отдельные таблицы из базы данных и объединм всё в одну таблицу, добавив нужные столбцы.

In [1]:
# импорт библиотек
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# путь к БД на компьютере
path_to_db_local = 'telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

In [3]:
# получение данных
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
    print('Загружены локальые данные.')
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
    print('Загружены данные с платформы.')
else:
    raise Exception('Файл с базой данных SQLite не найден!')

Загружены локальые данные.


In [4]:
# извлечение данных с помощью SQL-запроса
if path_to_db:
    # создание подключения к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    # запрос
    query = """
    SELECT 
    u.user_id,
    u.lt_day,
    CASE 
        WHEN u.lt_day <= 365 THEN 'new'
        ELSE 'old'
    END AS is_new,
    u.age,
    CASE 
        WHEN u.gender_segment = 0 THEN 'м'
        WHEN u.gender_segment = 1 THEN 'ж'
    END AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    u.nps_score,
    l.country,
    l.city,
    a.title AS age_segment,
    ts.title AS traffic_segment,
    ls.title AS lifetime_segment,
    CASE 
        WHEN u.nps_score BETWEEN 9 AND 10 THEN 'cторонники'
        WHEN u.nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
        ELSE 'критики'
    END AS nps_group
 
    FROM
        "user" u
    LEFT JOIN
        location l 
        ON u.location_id = l.location_id

    LEFT JOIN
        age_segment a
        ON u.age_gr_id = a.age_gr_id

    LEFT JOIN
        traffic_segment ts 
        ON u.tr_gr_id = ts.tr_gr_id 
        
    LEFT JOIN
        lifetime_segment ls 
        ON u.lt_gr_id = ls.lt_gr_id
        """  
    
    
    # создаём датафреймы по данным запроса
    df = pd.read_sql(query, engine)
    print('Данные загрузены.')

Данные загрузены.


In [5]:
# проверка заголовков столбцов получившейся таблицы
df.columns

Index(['user_id', 'lt_day', 'is_new', 'age', 'gender_segment', 'os_name',
       'cpe_type_name', 'nps_score', 'country', 'city', 'age_segment',
       'traffic_segment', 'lifetime_segment', 'nps_group'],
      dtype='object')

In [6]:
# вывод нескольких строк для примера
df.sample(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,nps_score,country,city,age_segment,traffic_segment,lifetime_segment,nps_group
241806,MI5YSX,1032,old,32.0,м,IOS,SMARTPHONE,2,Россия,Москва,03 25-34,09 25-30,07 25-36,критики
142549,HCL8W2,4023,old,30.0,м,ANDROID,SMARTPHONE,8,Россия,Екатеринбург,03 25-34,05 5-10,08 36+,нейтралы
182746,JFYPSD,2007,old,28.0,м,ANDROID,SMARTPHONE,10,Россия,Тверь,03 25-34,13 45-50,08 36+,cторонники


In [7]:
# вывод общей информации по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   nps_score         502493 non-null  int64  
 8   country           502493 non-null  object 
 9   city              502493 non-null  object 
 10  age_segment       502493 non-null  object 
 11  traffic_segment   502493 non-null  object 
 12  lifetime_segment  502493 non-null  object 
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


Определим, как много пропусков в столбцах с пропущенными данными.

In [8]:
# выборка нужных столбцов
null_cols = df.isnull().sum()
null_cols = null_cols[null_cols > 0]

# форматирование данных
percent_nulls = round(null_cols/len(df)*100, 2)
percent_nulls = percent_nulls.apply(lambda x: "{:.2f}%".format(x))

# вывод данных на экран
print("Процент пропусков в каждом столбце с пропусками:\n", percent_nulls.to_string(dtype=False), sep="")

Процент пропусков в каждом столбце с пропусками:
age               0.11%
gender_segment    0.26%


Восстановить такие пропуски не получится, оставим как есть.

## Предобработка данных

### Изменение типа данных и обработка пропусков

Проверим данные в столбце 'age' (сейчас там 'float', а должно быть 'int')

In [9]:
# вывод уникальных значений
df['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

In [10]:
# изменение типа данных с заменой пропусков
df['age'] = df['age'].fillna(0).astype(int)

In [11]:
# повторный вывод для проверки
df['age'].unique()

array([45, 53, 57, 44, 24, 42, 35, 36, 54, 39, 21, 27, 60, 34, 47, 37, 43,
       33, 31, 25, 51, 28, 41, 40, 46, 48, 32, 30, 52, 59, 26, 50, 62, 29,
       55, 22, 38, 56, 23, 49, 66, 74, 75, 17, 65, 64, 69, 58, 20, 19, 80,
       70, 81, 63, 67, 68, 72, 15, 79, 18, 73,  0, 14, 71, 61, 16, 77, 13,
       76, 10, 78, 12, 82, 11, 83, 89, 84, 85, 87, 86])

### Проверка и обработка дубликатов

Проверим, есть ли в датасете дубликаты.

In [12]:
# подсчёт полных дубликатов строк
print('Полных дубликатов в датасете: {}'.format(df.duplicated().sum()))

# подсчёт дубликтов user_id
print('Дубликаты user_id: {}'
      .format(df['user_id'].duplicated().sum()))

Полных дубликатов в датасете: 0
Дубликаты user_id: 0


Дубликатов нет, можно двигаться дальше.

### Логическая проверка столбцов

#### Лайфтайм-сегмент и новые-старые клиенты

Проверим, соответствует ли значение столбца 'is_new' категории лайфтайм-сегмента.  
Значение `new` должно соответствовать следующим категориям:
- '05 7-12',
- '04 4-6',
- '02 2',
- '03 3',
- '01 1'

In [13]:
# вывод фильтра по столбцу 'is_new'
df.query('is_new == "new"')['lifetime_segment'].unique()

array(['05 7-12', '04 4-6', '02 2', '03 3', '06 13-24', '08 36+', '01 1'],
      dtype=object)

Есть несоответствия ('06 13-24', '08 36+'). Изучим их.

In [14]:
# вывод ошибочных значений
print('Ошибочные значения для "06 13-24":',
      df.query('is_new == "new" and lifetime_segment == "06 13-24"')['lt_day'].unique())

print('Ошибочные значения для "08 36+":',
      df.query('is_new == "new" and lifetime_segment == "08 36+"')['lt_day'].unique())

Ошибочные значения для "06 13-24": [364 363 361 362 365]
Ошибочные значения для "08 36+": [ -8  -2  -4 -21 -13  -6 -12 -11  -1  -7]


Тут явная ошибка в данных. Заменим ошибочные значения на медианы для каждой категории.

In [15]:
# определение медианы
median = df.query('lifetime_segment == "08 36+"')['lt_day'].median()
# замена
df.loc[df['lt_day'] < 0, 'lt_day'] = median
df.loc[(df['lifetime_segment'] == "08 36+") & (df['is_new'] == "new"), 'is_new'] = 0

In [16]:
# определение медианы
median = df.query('lifetime_segment == "06 13-24"')['lt_day'].median()
# замена
df.loc[(df['lt_day'] < 366) & (df['lifetime_segment'] == "06 13-24") &
       (df['is_new'] == "new"), 'lt_day'] = median
df.loc[(df['lifetime_segment'] == "06 13-24") & (df['is_new'] == "new"), 'is_new'] = 0

Проверим правильность данных.

In [17]:
# вывод фильтра для проверки
df.query('is_new == "new"')['lifetime_segment'].unique()

array(['05 7-12', '04 4-6', '02 2', '03 3', '01 1'], dtype=object)

#### Возрастной сегмент и возраст

Посмотрим на значения столбца с возрастным сегментом.

In [18]:
# получение уникальных значений столбца с возрастными сегментами и сортировка
age_segments = sorted(df['age_segment'].unique())

# вывод каждого возрастного сегмента с уникальными значениями возрастов для сегмента
for segment in age_segments:
    print('ВОЗРАСТНОЙ СЕГМЕНТ:', segment)
    print(sorted(df[df['age_segment'] == segment]['age'].unique()),'\n')

ВОЗРАСТНОЙ СЕГМЕНТ: 01 до 16
[10, 11, 12, 13, 14, 15] 

ВОЗРАСТНОЙ СЕГМЕНТ: 02 16-24
[16, 17, 18, 19, 20, 21, 22, 23, 24] 

ВОЗРАСТНОЙ СЕГМЕНТ: 03 25-34
[25, 26, 27, 28, 29, 30, 31, 32, 33, 34] 

ВОЗРАСТНОЙ СЕГМЕНТ: 04 35-44
[35, 36, 37, 38, 39, 40, 41, 42, 43, 44] 

ВОЗРАСТНОЙ СЕГМЕНТ: 05 45-54
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54] 

ВОЗРАСТНОЙ СЕГМЕНТ: 06 55-64
[55, 56, 57, 58, 59, 60, 61, 62, 63, 64] 

ВОЗРАСТНОЙ СЕГМЕНТ: 07 66 +
[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89] 

ВОЗРАСТНОЙ СЕГМЕНТ: 08 n/a
[0] 



Возраст 65 лет приходится на возрастной сегмент '07 66 +', а в названиях сегментов нет диапазона, куда мог бы попасть этот возраст.

Изменим название сегмента '07 66 +' на '07 65 +', так как все значения 65 лет входят в категория '07 66 +'.

In [19]:
# изменение названия сегмента
df['age_segment'] = df['age_segment'].replace('07 66 +', '07 65 +')

In [20]:
# вывод всех значений столбца с сегментами (проверка)
df['age_segment'].unique()

array(['05 45-54', '06 55-64', '04 35-44', '02 16-24', '03 25-34',
       '07 65 +', '01 до 16', '08 n/a'], dtype=object)

#### Географические данные

Посмотрим, какие значения встречаются в столбцах с географическими данными.

In [21]:
# вывод информации
print(df['country'].unique())
print(df['city'].unique())

['Россия']
['Уфа' 'Киров' 'Москва' 'РостовнаДону' 'Рязань' 'Омск' 'СанктПетербург'
 'Волгоград' 'Тольятти' 'Казань' 'Самара' 'Красноярск' 'Екатеринбург'
 'Калуга' 'Краснодар' 'Иркутск' 'Пермь' 'Владимир' 'Ижевск' 'Тюмень'
 'Оренбург' 'НижнийНовгород' 'Брянск' 'Челябинск' 'Астрахань' 'Сургут'
 'Тверь' 'Новосибирск' 'НабережныеЧелны' 'Махачкала' 'Воронеж' 'Курск'
 'Владивосток' 'Балашиха' 'Пенза' 'Калининград' 'Тула' 'Саратов'
 'Кемерово' 'Белгород' 'Барнаул' 'Чебоксары' 'Архангельск' 'Томск'
 'Ярославль' 'Ульяновск' 'Хабаровск' 'Грозный' 'Ставрополь' 'Липецк'
 'Новокузнецк' 'Якутск' 'УланУдэ' 'Сочи' 'Иваново' 'НижнийТагил'
 'Смоленск' 'Волжский' 'Магнитогорск' 'Чита' 'Череповец' 'Саранск']


Создадим дополнительный столбец с английскими эквивалентами названий городов.

In [22]:
# создание словаря
city_dict = {'Уфа': 'Ufa', 'Киров': 'Kirov', 'Москва': 'Moscow', 'РостовнаДону': 'Rostov-on-Don',
             'Рязань': 'Ryazan', 'Омск': 'Omsk','СанктПетербург': 'SaintPetersburg',
             'Волгоград': 'Volgograd', 'Тольятти': 'Tolyatti', 'Казань': 'Kazan',
             'Самара': 'Samara','Красноярск': 'Krasnoyarsk', 'Екатеринбург': 'Yekaterinburg',
             'Калуга': 'Kaluga', 'Краснодар': 'Krasnodar', 'Иркутск': 'Irkutsk',
             'Пермь': 'Perm', 'Владимир': 'Vladimir', 'Ижевск': 'Izhevsk', 'Тюмень': 'Tyumen',
             'Оренбург': 'Orenburg','НижнийНовгород': 'Nizhny Novgorod', 'Брянск': 'Bryansk',
             'Челябинск': 'Chelyabinsk', 'Астрахань': 'Astrakhan', 'Сургут': 'Surgut',
             'Тверь': 'Tver', 'Новосибирск': 'Novosibirsk', 'НабережныеЧелны': 'Naberezhnye Chelny',
             'Махачкала': 'Makhachkala', 'Воронеж': 'Voronezh','Курск': 'Kursk',
             'Владивосток': 'Vladivostok', 'Балашиха': 'Balashikha', 'Пенза': 'Penza',
             'Калининград': 'Kaliningrad', 'Тула': 'Tula','Саратов': 'Saratov', 'Кемерово': 'Kemerovo',
             'Белгород': 'Belgorod', 'Барнаул': 'Barnaul', 'Чебоксары': 'Cheboksary',
             'Архангельск': 'Arkhangelsk', 'Томск': 'Tomsk', 'Ярославль': 'Yaroslavl',
             'Ульяновск': 'Ulyanovsk', 'Хабаровск': 'Khabarovsk', 'Грозный': 'Grozny',
             'Ставрополь': 'Stavropol', 'Липецк': 'Lipetsk', 'Новокузнецк': 'Novokuznetsk',
             'Якутск': 'Yakutsk', 'УланУдэ': 'Ulan-Ude', 'Сочи': 'Sochi', 'Иваново': 'Ivanovo',
             'НижнийТагил': 'Nizhny Tagil', 'Смоленск': 'Smolensk', 'Волжский': 'Volzhsky',
             'Магнитогорск': 'Magnitogorsk', 'Чита': 'Chita', 'Череповец': 'Cherepovets', 'Саранск': 'Saransk'}

# добавление столбца
df['city_en'] = df['city'].map(city_dict)

In [23]:
# проверка
df[['city', 'city_en']].sample(n=3)

,city,city_en
408473,Казань,Kazan
288367,Москва,Moscow
333439,Красноярск,Krasnoyarsk


__Данные предобработаны, можно выгружать получившуюся таблицу для работы в Tableu.__

## Выгрузка данных

In [24]:
# сохранение таблицы на компьютер для дальнейшей работы
df.to_csv('df_telecomm.csv', index=False)

## Презентация

Дашборд доступен по ссылке: <https://public.tableau.com/views/telecommNPSproject/sheet0?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link>